# 用变分推断估计高斯分布的参数
在上一篇中，我们通过对高斯分布的均值$\mu|\tau$和精度 $\tau$分别指定不同共轭先验：
$$ x_i|\mu,\tau\stackrel{ i.i.d.}{\sim}\mathcal{N}(\mu,\tau^{-1})=(\frac{\tau}{2\pi})^{n/2}\exp(-\frac{\tau}{2}\sum_{i=1}^n(x_i-\mu)^2) \\ \mu|\tau \sim\mathcal{N}(\mu_0,(\lambda_0 \tau)^{-1})\propto \exp(-\frac{\lambda_0 \tau}{2}(\mu-\mu_0)^2)\\ \tau\sim Ga(\alpha_0,\beta_0)\propto \tau^{\alpha_0-1}\exp(-\beta_0\tau)$$
通过一系列计算最终得到了后验分布的解析解：
$$ \tau|x\sim Ga(\alpha_0+\frac{n}{2},\beta_0+\frac{1}{2}\sum_{i=1}^n (x_i-\bar{x})^2+\frac{n\lambda_0(\bar{x}-\mu_0)^2}{2(n+\lambda_0)})\\\mu|x,\tau\sim\mathcal{N}(\frac{n\bar{x}+\lambda_0\mu_0}{n+\lambda_0},\big((n+\lambda_0)\tau\big)^{-1})$$

这次我们假装不知道如何估计参数，转而采用变分推断估计$\mu$和$\tau$
变分推断中，我们需要把隐变量进行分解：
$$q(\mu,\tau)=q_\mu(\mu)q_\tau(\tau)$$
均值的变分推断的迭代更新式为：
$$\begin{aligned}ln\,(q^*_\mu(\mu))&=\mathbb{E}_{q_\tau(\tau)}[ln\,p(D,\mu,\tau)]+const\\&=\mathbb{E}_{q_\tau(\tau)}[ln\,p(D|\mu,\tau)+ln\,p(\mu|\tau)+\underbrace{ln\,p(\tau)}_{\mbox{constant}}]+const\\&=\mathbb{E}_{q_\tau(\tau)}[ln\,p(D|\mu,\tau)+ln\,p(\mu|\tau)]+const\\&=-\frac{\mathbb{E}_{q_\tau(\tau)}(\tau)}{2}\bigg\{\sum_{i=1}^n(x_i-\mu)^2+\lambda_0(\mu-\mu_0)^2\bigg\}+const\\&=-\frac{\mathbb{E}_{q_\tau(\tau)}(\tau)}{2}\bigg\{n(\mu-\bar{x})^2+\sum_{i=1}^n(x_i-\bar{x})^2+\lambda_0(\mu-\mu_0)^2\bigg\}+const\\&=-\frac{\mathbb{E}_{q_\tau(\tau)}(\tau)}{2}\bigg\{n\mu^2-2n\bar{x}\mu+\lambda_0 \mu^2-2\lambda_0\mu_0\mu\bigg\}+const\\&=-\frac{\mathbb{E}_{q_\tau(\tau)}(\tau)}{2}\bigg\{(n+\lambda_0)\mu^2-2(n\bar{x}+\lambda_0\mu_0)\mu\bigg\}+const\\&=-\frac{\mathbb{E}_{q_\tau(\tau)}(\tau)}{2}\bigg\{(n+\lambda_0)\Big[(\mu-\frac{n\bar{x}+\lambda_0\mu_0}{n+\lambda_0})^2+(\frac{n\bar{x}+\lambda_0\mu_0}{n+\lambda_0})^2\Big]\bigg\}+const\\&=-\frac{(n+\lambda_0)\mathbb{E}_{q_\tau(\tau)}(\tau)}{2}(\mu-\frac{n\bar{x}+\lambda_0\mu_0}{n+\lambda_0})^2+const\end{aligned}$$
其中$\mathbb{E}_{q_\tau(\tau)}[ln\,p(\tau)]$由于积分把$\tau$积没了，可以视为常数，放到const里面，所有含有$\tau$的项可以用相同方法处理。
因此
$$ q^*_\mu(\mu)\propto \exp(-\frac{(n+\lambda_0)\mathbb{E}_{q_\tau(\tau)}(\tau)}{2}(\mu-\frac{n\bar{x}+\lambda_0\mu_0}{n+\lambda_0})^2)\propto \mathcal{N}(\mu|\frac{n\bar{x}+\lambda_0\mu_0}{n+\lambda_0},\big((n+\lambda_0)\mathbb{E}_{q_\tau(\tau)}(\tau)\big)^{-1})$$

精度的迭代更新式为：
$$\begin{aligned}ln\,(q^*_\tau(\tau))&=\mathbb{E}_{q_\mu(\mu)}[ln\,p(D,\mu,\tau)]+const\\&=\mathbb{E}_{q_\mu(\mu)}[ln\,p(D|\mu,\tau)+ln\,p(\mu|\tau)+ln\,p(\tau)]+const\\&=\mathbb{E}_{q_\mu(\mu)}[\frac{n}{2}ln\,(\tau)-\frac{\tau}{2}\sum_{i=1}^n(x_i-\mu)^2-\frac{\lambda_0\tau}{2}(\mu-\mu_0)^2+(\alpha_0-1)ln\,(\tau)-\beta_0\tau]+const\\&=(\underbrace{\alpha_0+\frac{n}{2}}_{\alpha_n}-1)ln\,(\tau)-\tau\Big\{\underbrace{\beta_0+\frac{1}{2}\mathbb{E}_{q_\mu(\mu)}\bigg(\sum_{i=1}^n(x_i-\mu)^2+\lambda_0(\mu-\mu_0)^2\bigg)}_{\beta_n}\Big\}+const\end{aligned}$$
$\beta_n$可以写为
$$\begin{aligned}\beta_n&=\beta_0+\frac{1}{2}\mathbb{E}_{q_\mu(\mu)}\bigg[\sum_{i=1}^n(x_i-\mu)^2+\lambda_0(\mu-\mu_0)^2\bigg]\\&=\beta_0+\frac{1}{2}\bigg\{\mathbb{E}_{q_\mu(\mu)}\bigg[n\mu^2-2n\bar{x}\mu+\lambda_0\mu^2-2\lambda_0\mu_0\mu\bigg]+\sum_{i=1}^n(x_i)^2+\lambda_0\mu_0^2\bigg\}\\&=\beta_0+\frac{1}{2}\bigg\{(n+\lambda_0)\mathbb{E}_{q_\mu(\mu)}[\mu^2]-2(n\bar{x}+\lambda_0\mu_0)\mathbb{E}_{q_\mu(\mu)}[\mu]+\sum_{i=1}^n(x_i)^2+\lambda_0\mu_0^2\bigg\}\end{aligned}$$

我们用上一轮迭代得到的$q^*_\mu(\mu)$计算$\mathbb{E}_{q_\mu(\mu)}[\mu^2]$和$\mathbb{E}_{q_\mu(\mu)}[\mu]$，再用这一轮得到的$q^*_\tau(\tau)$计算下一轮的$(n+\lambda_0)\mathbb{E}_{q_\tau(\tau)}(\tau)$

